In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AR


H=5 #trading horizon
file1 = 'Constant_Maturity_EDs.csv'
ed_data_constant_maturity = pd.read_csv(file1, index_col= 0)
ed_data_constant_maturity.index = pd.to_datetime(ed_data_constant_maturity.index)


sample_A = ed_data_constant_maturity.ix['2011-06-01':'2014-01-31',:]
sample_B = ed_data_constant_maturity.ix['2014-06-15':'2015-06-15',:]
sample_C = ed_data_constant_maturity.ix['2015-06-15':'2016-06-14',:]
 
def halflife(z):
    z_lag = z.shift(1)
    z_lag.ix[0] = z_lag.ix[1]
    z_ret = z - z_lag
    z_ret.ix[0] = z_ret.ix[1]
    z_lag2 = sm.add_constant(z_lag)
    model = sm.OLS(z_ret,z_lag2)
    res = model.fit()
    halflife = round(-np.log(2) / res.params[1],0)
    return halflife

def Cointegration(a1,a2):
    coint_= sm.OLS(a1,a2).fit()
    coint=pd.Series(a1) - pd.Series(coint_.params.ix[0]*a2)
    return coint,coint_

def Signal1(a):
    m = AR(a).fit(1)
    predict=m.predict()
    return predict,m
    
def Signal2(a,alpha):
    signal2=a-pd.Series(a).ewm(alpha=alpha).mean()
    predict1,error1=Signal1(signal2)
    return predict1
                


cointA23, paramA23 = Cointegration(sample_A.ix[:,'2.0Y'],sample_A.ix[:,'3.0Y'])
cointA34,paramA34 = Cointegration(sample_A.ix[:,'3.0Y'],sample_A.ix[:,'4.0Y'])
cointA45,paramA45 = Cointegration(sample_A.ix[:,'4.0Y'],sample_A.ix[:,'4.75Y'])

signal1A23,a23 =  Signal1(cointA23)
signal1A34,a34 =  Signal1(cointA23)
signal1A45,a45 =  Signal1(cointA23)

HLA_23=halflife(signal1A23)
HLA_34=halflife(signal1A34)
HLA_45=halflife(signal1A45)


alpha= 1- np.exp(np.log(0.5)/10)
signal2A23=Signal2(cointA23,alpha)
signal2A34=Signal2(cointA34,alpha)
signal2A45=Signal2(cointA45,alpha)

 
coint23B,mod23B=Cointegration(sample_B.ix[:,'2.0Y'],sample_B.ix[:,'3.0Y'])
coint34B,mod34B=Cointegration(sample_B.ix[:,'3.0Y'],sample_B.ix[:,'4.0Y'])
coint45B,mod45B=Cointegration(sample_B.ix[:,'4.0Y'],sample_B.ix[:,'4.75Y'])

def forecast(t,param):
    beta=param.params[1]
    c=param.params[0]
    t_for=(beta**H)*t+ c*((1-beta**H)/(1-beta))
    return t_for
    

"""Algorithm:
    
    By using the Gaussian Mixture function from the sklearn module, and varying the
    values of different weights parameter, we can find the optimal thetas.
    
    pseudo code for GMM:
        we select random thetas, find the variance of the dataset. Then we use the 
        expectation maximisation algorithm.
        guess parameters,repeat the following until convergence:
            compute the expectation 
            Estimate the new parameters"""
            
"""References:
        Machine Learning: An algorithmic perspective"""
        
    




nan nan nan


/Users/Hatim/anaconda/lib/python3.6/site-packages/numpy/linalg/linalg.py:1546: RuntimeWarning: invalid value encountered in greater
  return sum(S > tol)
/Users/Hatim/anaconda/lib/python3.6/site-packages/statsmodels/base/data.py:503: FutureWarning: TimeSeries is deprecated. Please use Series
  return TimeSeries(result, index=self.predict_dates)


'References:\n        Machine Learning: An algorithmic perspective'